### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve,scorer
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [3]:
# X_test = pd.read_csv("X_test.csv")
# y_test = pd.read_csv("y_test.csv")

features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
            'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
target = 'income'

df = pd.read_csv('data/adult.data', sep=',', index_col=False, names=features + [target])
df_test = pd.read_csv('data/adult.test', sep=',', index_col=False, names=features + [target])
# Одна строка в данных "битая", удалим.
df_test=df_test[df_test['age']!='|1x3 Cross validator']

In [4]:
X = df.drop(columns=['income', 'fnlwgt', 'native-country', 'race', 'sex'])
y = df['income']

X_test = df_test.drop(columns=['income', 'fnlwgt', 'native-country', 'race', 'sex'])
y_test = df_test['income']

In [5]:
X_test.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,capital-gain,capital-loss,hours-per-week
1,25,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,0.0,0.0,40.0
2,38,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,0.0,0.0,50.0
3,28,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,0.0,0.0,40.0


In [6]:
with open('xgboost_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [7]:
predictions = pipeline.predict_proba(X_test)
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [8]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:])

0.8383121299922364

In [29]:
X_test.groupby('hours-per-week').count().T

hours-per-week,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,84.0,85.0,86.0,88.0,89.0,90.0,92.0,96.0,98.0,99.0
age,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
workclass,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
education,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
education-num,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
marital-status,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
occupation,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
relationship,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
capital-gain,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
capital-loss,7,21,20,30,35,28,19,73,9,147,...,27,4,2,2,1,13,2,4,3,52
